# Control Cinemático RR

Manipulador planar de dos articulaciones rotacionales siguiendo una trayectoria circular.

## 1. Configuración

In [ ]:
import sys
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 60

ROOT = pathlib.Path().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from notebooks.control.utils import time_vector, circular_task_reference, plot_tracking

In [ ]:
sim_config = {"t0": 0.0, "tf": 20.0, "n_steps": 600}
radius = 0.1
center = (0.25, 0.15)
omega = 2 * np.pi / (sim_config["tf"] - sim_config["t0"])

t, dt = time_vector(sim_config)
Xd, Vd = circular_task_reference(radius, center, omega, t)

L1 = 0.18
L2 = 0.18

## 2. Modelo cinemático

In [ ]:
def forward_kinematics(q: np.ndarray) -> np.ndarray:
    q1, q2 = q
    x = L1 * np.cos(q1) + L2 * np.cos(q1 + q2)
    y = L1 * np.sin(q1) + L2 * np.sin(q1 + q2)
    return np.array([x, y])


def jacobian(q: np.ndarray) -> np.ndarray:
    q1, q2 = q
    s1 = np.sin(q1)
    c1 = np.cos(q1)
    s12 = np.sin(q1 + q2)
    c12 = np.cos(q1 + q2)
    return np.array(
        [
            [-L1 * s1 - L2 * s12, -L2 * s12],
            [L1 * c1 + L2 * c12, L2 * c12],
        ]
    )

## 3. Control cinemático

In [ ]:
Kp = np.diag([6.0, 6.0])
q = np.zeros((2, t.size))
q[:, 0] = np.array([np.pi / 4, -np.pi / 6])
X = np.zeros((2, t.size))

for k in range(t.size - 1):
    X[:, k] = forward_kinematics(q[:, k])
    error = Xd[:, k] - X[:, k]
    v_ref = Vd[:, k] + Kp @ error
    J = jacobian(q[:, k])
    dq = np.linalg.pinv(J) @ v_ref
    q[:, k + 1] = q[:, k] + dq * dt

X[:, -1] = forward_kinematics(q[:, -1])

## 4. Verificación del error

In [ ]:
tracking_error = np.linalg.norm(Xd - X, axis=0)
max_error = float(tracking_error.max())
print(f"Error máximo de seguimiento: {max_error:.4f} m")
assert max_error < 0.02, "El error de seguimiento excede el umbral de 2 cm"

## 5. Trayectorias

In [ ]:
fig = plot_tracking(X, Xd, t)
fig

## 6. Animación

In [ ]:
def arm_points(q: np.ndarray) -> np.ndarray:
    q1, q2 = q
    base = np.array([0.0, 0.0])
    joint = L1 * np.array([np.cos(q1), np.sin(q1)])
    end = joint + L2 * np.array([np.cos(q1 + q2), np.sin(q1 + q2)])
    return np.stack((base, joint, end), axis=1)


fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect("equal", adjustable="box")
ax.set_xlim(-0.2, 0.6)
ax.set_ylim(-0.2, 0.6)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.set_title("Control cinemático RR")
ax.plot(Xd[0], Xd[1], "--", color="tab:orange", label="Trayectoria deseada")
arm_line, = ax.plot([], [], "o-", lw=2, color="tab:blue", label="Robot")
ax.legend(loc="upper right")

frames = range(0, t.size, 2)


def init():
    arm_line.set_data([], [])
    return (arm_line,)


def animate(i):
    pts = arm_points(q[:, i])
    arm_line.set_data(pts[0], pts[1])
    return (arm_line,)


anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=frames, interval=50, blit=True
)
HTML(anim.to_jshtml())